In [5]:
import pandas as pd
import os
from datetime import datetime
import openpyxl  # Motor recomendado para escribir archivos Excel (.xlsx)

In [6]:
V_Periodo_Actual = "2025-04-01"
Dia = "2025-03-01"
Mora = "2025-02-01"

In [7]:
#rutas Capresoca 
R_MaestroAdres = r"C:\DIANAB\ASEGURAMIENTO 2025\3. GESTION DE CARTERA\Automatización\Adicionales\EPSC25MC0027032025.TXT" # Cambiar nombre
R_Relaciones_Laborales_SIE = r"C:\DIANAB\ASEGURAMIENTO 2025\3. GESTION DE CARTERA\Automatización\Adicionales\Reporte_Afiliados Contributivo Relaciones Laborales_2025_04_02.csv" # Cambiar nombre

R_Pila3047 = r"C:\DIANAB\ASEGURAMIENTO 2025\3. GESTION DE CARTERA\Automatización\Pila consiliada ADRES\Pila_Unificado_Con_Aportante_2018_2025.TXT"
R_Pila_I_SIE = r"C:\DIANAB\ASEGURAMIENTO 2025\3. GESTION DE CARTERA\Automatización\Pila I"
R_Pila_IP_SIE = r"C:\DIANAB\ASEGURAMIENTO 2025\3. GESTION DE CARTERA\Automatización\Pila IP"

In [8]:
# Rutas de salida Capresoca 
R_Salida_Pila_SIE_i = r"C:\DIANAB\ASEGURAMIENTO 2025\3. GESTION DE CARTERA\Automatización\df_pila_i_sie.xlsx"
#R_Salida_Relaciones_Laborales = r"C:\Users\osmarrincon\OneDrive - uniminuto.edu\Capresoca\AlmostClear\Procesos BDUA\Contributivo\Compensación\_Pila_SIE\SIE\Relaciones laborales.txt"

In [9]:
import glob

# Obtener la lista de archivos .txt en la ruta especificada
file_list = glob.glob(R_Pila_I_SIE + "/*.TXT")
file_list_IP = glob.glob(R_Pila_IP_SIE + "/*.TXT")

# Leer y concatenar todos los archivos .txt en un solo dataframe
df_pila_i_sie = pd.concat((pd.read_csv(file, sep='|', encoding='ANSI') for file in file_list), ignore_index=True)
df_pila_iP_sie = pd.concat((pd.read_csv(file, sep='|', encoding='ANSI', header=None) for file in file_list_IP), ignore_index=True)
DF_MC_Adres = pd.read_csv(R_MaestroAdres, sep=',', encoding='ansi', header=None)
df_Pila_3047 = pd.read_csv(R_Pila3047, sep=',', encoding='UTF-16')
df_Relaciones_Laborales_SIE = pd.read_csv(R_Relaciones_Laborales_SIE, sep=';', encoding='ansi')

C:\Users\dianaborda\AppData\Local\Temp\ipykernel_22384\3879205004.py:8: DtypeWarning: Columns (79,80,81,82) have mixed types. Specify dtype option on import or set low_memory=False.
  df_pila_i_sie = pd.concat((pd.read_csv(file, sep='|', encoding='ANSI') for file in file_list), ignore_index=True)
C:\Users\dianaborda\AppData\Local\Temp\ipykernel_22384\3879205004.py:8: DtypeWarning: Columns (79,80) have mixed types. Specify dtype option on import or set low_memory=False.
  df_pila_i_sie = pd.concat((pd.read_csv(file, sep='|', encoding='ANSI') for file in file_list), ignore_index=True)
C:\Users\dianaborda\AppData\Local\Temp\ipykernel_22384\3879205004.py:8: DtypeWarning: Columns (43,44,79,80) have mixed types. Specify dtype option on import or set low_memory=False.
  df_pila_i_sie = pd.concat((pd.read_csv(file, sep='|', encoding='ANSI') for file in file_list), ignore_index=True)
C:\Users\dianaborda\AppData\Local\Temp\ipykernel_22384\3879205004.py:8: DtypeWarning: Columns (29,43,44,79,80) h

In [10]:
# Reemplazar los valores 'X' por 1 en todo el dataframe df_pila_iP_sie
df_pila_iP_sie = df_pila_iP_sie.replace('X', 1)
df_Pila_3047 = df_Pila_3047.replace('X', 1)

C:\Users\dianaborda\AppData\Local\Temp\ipykernel_22384\1628223100.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_pila_iP_sie = df_pila_iP_sie.replace('X', 1)
C:\Users\dianaborda\AppData\Local\Temp\ipykernel_22384\1628223100.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_Pila_3047 = df_Pila_3047.replace('X', 1)


In [11]:
df_pila_i_sie['origen'] = 'Pila_I'
df_pila_iP_sie['origen'] = 'Pila_IP'
df_Pila_3047['origen'] = 'Pila_3047'
print(f"Número de columnas en df_pila_i_sie: {df_pila_i_sie.shape[1]}")
print(f"Número de columnas en df_pila_iP_sie: {df_pila_iP_sie.shape[1]}")
print(f"Número de columnas en df_Pila_3047: {df_Pila_3047.shape[1]}")

Número de columnas en df_pila_i_sie: 92
Número de columnas en df_pila_iP_sie: 92
Número de columnas en df_Pila_3047: 45


In [12]:
# Mostrar la cantidad de registros antes de la unificación
print(f"Cantidad de registros en df_pila_i_sie antes de la unificación: {len(df_pila_i_sie)}")
print(f"Cantidad de registros en df_pila_iP_sie antes de la unificación: {len(df_pila_iP_sie)}")

# Validar que ambos dataframes tengan la misma cantidad de columnas
if df_pila_iP_sie.shape[1] == df_pila_i_sie.shape[1]:
    # Asignar el mismo nombre de columnas de df_pila_i_sie a df_pila_iP_sie
    df_pila_iP_sie.columns = df_pila_i_sie.columns
    
    # Unificar ambos dataframes
    df_pila_i_sie = pd.concat([df_pila_i_sie, df_pila_iP_sie], ignore_index=True)
    
    # Validar que la unificación se haya realizado correctamente
    if len(df_pila_i_sie) > len(df_pila_i_sie) - len(df_pila_iP_sie):
        print("Unificación realizada correctamente.")
    else:
        print("Error en la unificación de los dataframes.")
    
    # Eliminar el dataframe df_pila_iP_sie
    del df_pila_iP_sie
else:
    print("Los dataframes no tienen la misma cantidad de columnas. No se puede realizar la unificación.")

# Mostrar la cantidad de registros después de la unificación
print(f"Cantidad de registros después de la unificación: {len(df_pila_i_sie)}")

Cantidad de registros en df_pila_i_sie antes de la unificación: 812502
Cantidad de registros en df_pila_iP_sie antes de la unificación: 18176
Unificación realizada correctamente.
Cantidad de registros después de la unificación: 830678


In [13]:
df_Datos_Complementarios = df_pila_i_sie

# Mostrar la cantidad de registros antes de filtrar
print(f"Cantidad de registros antes de filtrar: {len(df_Datos_Complementarios)}")

# Convertir la columna 'Perido Pago' a tipo datetime
df_Datos_Complementarios['Perido Pago'] = pd.to_datetime(df_Datos_Complementarios['Perido Pago'], format='%Y-%m')

# Ordenar el dataframe por las columnas especificadas y por 'Perido Pago' en orden descendente
df_pila_df_Datos_Complementariosi_sie = df_Datos_Complementarios.sort_values(by=['Razón Social Aportante', 'N° Identificación Aportante', 'Perido Pago'], ascending=[True, True, False])

# Eliminar duplicados manteniendo solo los registros con el periodo máximo
df_Datos_Complementarios = df_Datos_Complementarios.drop_duplicates(subset=['Razón Social Aportante', 'N° Identificación Aportante'], keep='first')

# Mostrar la cantidad de registros después de filtrar
print(f"Cantidad de registros después de filtrar: {len(df_Datos_Complementarios)}")

Cantidad de registros antes de filtrar: 830678
Cantidad de registros después de filtrar: 28651


In [14]:
# Mostrar las primeras filas del dataframe
print(f"Cantidad de registros Pila SIE: {len(df_pila_i_sie)}")

print(DF_MC_Adres.columns)
print(f"Cantidad de registros MC ADRES: {len(DF_MC_Adres)}")

Cantidad de registros Pila SIE: 830678
Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37],
      dtype='int64')
Cantidad de registros MC ADRES: 21535


In [15]:
# Seleccionar solo las columnas especificadas
columns_to_keep = [
    4, 5, 6, 7, 8, 9, 12, 18, 19, 28, 36 
]

# Filtrar el dataframe para mantener solo las columnas especificadas
DF_MC_Adres = DF_MC_Adres[columns_to_keep]
DF_MC_Adres = DF_MC_Adres[
    (DF_MC_Adres[28] != "AF") | (DF_MC_Adres[28].isna())
]

# Asignar nombres a las columnas seleccionadas
DF_MC_Adres.columns = [
    'Tp_Do', 'No_Do', '1A', '2A', '1N', '2N', 'Tp_Afiliado', 'Departamento', 'Municipio', 'Estado_ADRES', 'Sisben'
]


# Mostrar las primeras filas del dataframe resultante
print(DF_MC_Adres.columns)
print(f"Cantidad de registros: {len(DF_MC_Adres)}")

Index(['Tp_Do', 'No_Do', '1A', '2A', '1N', '2N', 'Tp_Afiliado', 'Departamento',
       'Municipio', 'Estado_ADRES', 'Sisben'],
      dtype='object')
Cantidad de registros: 21251


In [16]:
df_pila_i_sie = df_pila_i_sie[
    (df_pila_i_sie['Correcciones'] != "A") | (df_pila_i_sie['Correcciones'].isna())
]

# Seleccionar solo las columnas especificadas
columns_to_keep = [
    'Razón Social Aportante', 'N° Identificación Aportante', 'Perido Pago', 'Fecha Pago',
    'Tipo Documento Cotizante', 'N° Identificación Cotizante', 'Tipo Cotizante', 'ING', 'RET', 'Días Cotizados', 
    'Ingreso Base Cotización', 'Cotización Obligatoria', 'Número Planilla', 'origen'
]
# Filtrar el dataframe para mantener solo las columnas especificadas
df_pila_i_sie = df_pila_i_sie[columns_to_keep]

In [17]:
df_Pila_3047 = df_Pila_3047.rename(columns={
    'Num_rad_planilla': 'Número Planilla',
    'perido_pago_del_aportante': 'Perido Pago',
    'fecha_pago': 'Fecha Pago',
    'Tipo_doc_cotizante': 'Tipo Documento Cotizante',
    'doc_cotizante' : 'N° Identificación Cotizante',
    'Tipo_cotizante': 'Tipo Cotizante',
    'ingreso': 'ING',
    'retiro': 'RET',
    'dias_cotizados': 'Días Cotizados',
    'ibc': 'Ingreso Base Cotización',
    'cotizacion_obligatoria':'Cotización Obligatoria',
    'Nit': 'N° Identificación Aportante',
    'Razon_Soacial': 'Razón Social Aportante'
})
print(df_Pila_3047.columns)

Index(['codigo_operador', 'Número Planilla', 'Perido Pago', 'Fecha Pago',
       'cod_eps', 'digito_nit', 'Tipo Documento Cotizante',
       'N° Identificación Cotizante', 'serial_benef_upc_adicional',
       'tipo_doc_benef_upc_adicional', 'Doc_benef_adicional', 'Tipo Cotizante',
       'subtipo_cotizante', 'tipo_pensionado', 'tipo_pension',
       'pension_compartida', 'extranjero_no_obligado_compensar',
       'colombiano_residente_eterior', 'cod_dept_ubicacion_laboral',
       'mun_ubicacion_laboral', 'apel_1', 'apel_2', 'nom_1', 'nom_2', 'ING',
       'RET', 'traslado-otra_administradora', 'traslado_a_otra_adminstradora',
       'variacion_permanente_salario', 'variacion_trnsitoria_salario',
       'suspension_temporal_de_contrato', 'vacaciones', 'Días Cotizados',
       'salario_basico', 'Ingreso Base Cotización', 'tarifa',
       'Cotización Obligatoria', 'valor_upc_adcional', 'Campo39', 'Campo40',
       'nombre_Archivo', 'Fecha_Archivo', 'N° Identificación Aportante',
       '

In [18]:
column_order = [
    'Razón Social Aportante', 'N° Identificación Aportante', 'Perido Pago', 'Fecha Pago',
    'Tipo Documento Cotizante', 'N° Identificación Cotizante', 'Tipo Cotizante', 'ING', 'RET', 'Días Cotizados', 
    'Ingreso Base Cotización', 'Cotización Obligatoria', 'Número Planilla', 'origen'
    ]
df_Pila_3047 = df_Pila_3047[column_order]

In [19]:
# Add new columns to df_sin_pagos
df_pila_i_sie['Tipo Cotizante'] = df_pila_i_sie['Tipo Cotizante'].fillna(0).astype('int64')


df_Pila_3047['Ingreso Base Cotización'] = df_Pila_3047['Ingreso Base Cotización'].astype('float64')
df_Pila_3047['N° Identificación Aportante'] = pd.to_numeric(df_Pila_3047['N° Identificación Aportante'], errors='coerce').fillna(0).astype('int64')
df_Pila_3047['Tipo Cotizante'] = df_Pila_3047['Tipo Cotizante'].astype('int64')
df_Pila_3047['ING'] = df_Pila_3047['ING'].fillna(0).astype('int64')
df_Pila_3047['RET'] = df_Pila_3047['RET'].fillna(0).astype('int64')
df_Pila_3047['Días Cotizados'] = df_Pila_3047['Días Cotizados'].fillna(0).astype('int64')
df_pila_i_sie['ING'] = df_pila_i_sie['ING'].fillna(0).astype('int64')
df_pila_i_sie['RET'] = df_pila_i_sie['RET'].fillna(0).astype('int64')


In [20]:
# Mostrar la cantidad de registros antes de la unión
print(f"Cantidad de registros en df_Pila_3047 antes de la unión: {len(df_Pila_3047)}")
print(f"Cantidad de registros en df_pila_i_sie antes de la unión: {len(df_pila_i_sie)}")

# Unir los dataframes uno debajo del otro
df_pila_i_sie = pd.concat([df_pila_i_sie, df_Pila_3047], ignore_index=True)

# Mostrar la cantidad de registros después de la unión
print(f"Cantidad de registros en df_pila_i_sie después de la unión: {len(df_pila_i_sie)}")

Cantidad de registros en df_Pila_3047 antes de la unión: 1350627
Cantidad de registros en df_pila_i_sie antes de la unión: 817450
Cantidad de registros en df_pila_i_sie después de la unión: 2168077


In [21]:
# Cambiar el nombre de las columnas especificadas
df_pila_i_sie = df_pila_i_sie.rename(columns={
    'Tipo Documento Cotizante': 'Tp_Do', 
    'N° Identificación Cotizante': 'No_Do'
})

print(f"Cantidad de registros Pila: {len(df_pila_i_sie)}")
# Realizar la unión de los dataframes
df_pila_i_sie = df_pila_i_sie.merge(DF_MC_Adres[['Tp_Do', 'No_Do', '1A', '2A', '1N', '2N', 'Tp_Afiliado', 'Departamento', 'Municipio', 'Estado_ADRES', 'Sisben']], on=['Tp_Do', 'No_Do'], how='left')
df_pila_i_sie = df_pila_i_sie[df_pila_i_sie['Estado_ADRES'].notna() & (df_pila_i_sie['Estado_ADRES'] == "AC")]


print(f"Cantidad de registros Pila: {len(df_pila_i_sie)}")
print(f"Cantidad de registros df_pila_i_sie: {len(df_pila_i_sie)}")

Cantidad de registros Pila: 2168077
Cantidad de registros Pila: 1000622
Cantidad de registros df_pila_i_sie: 1000622


In [22]:
# Mostrar la cantidad de registros antes de filtrar
print(f"Cantidad de registros antes de filtrar: {len(df_pila_i_sie)}")

# Convertir la columna 'Días Cotizados' a tipo numérico
df_pila_i_sie['Días Cotizados'] = pd.to_numeric(df_pila_i_sie['Días Cotizados'], errors='coerce')

# Sumar los días cotizados por cada ID1
df_pila_i_sie['Días Cotizados'] = df_pila_i_sie.groupby(
    ["N° Identificación Aportante", "Tp_Do", "No_Do", 'Perido Pago']
)['Días Cotizados'].transform('sum')

# Si en la columna ING hay un 1, todos los registros del mismo grupo quedan con 1
df_pila_i_sie['ING'] = df_pila_i_sie.groupby(
    ["N° Identificación Aportante", "Tp_Do", "No_Do", 'Perido Pago']
)['ING'].transform('max')

# Si en la columna RET hay un 1, todos los registros del mismo grupo quedan con 1
df_pila_i_sie['RET'] = df_pila_i_sie.groupby(
    ["N° Identificación Aportante", "Tp_Do", "No_Do", 'Perido Pago']
)['RET'].transform('max')



# Mostrar las primeras filas del dataframe resultante
print(f"Cantidad de registros después de la agregación: {len(df_pila_i_sie)}")

Cantidad de registros antes de filtrar: 1000622
Cantidad de registros después de la agregación: 1000622


In [23]:
# Convertir la columna 'Perido Pago' a datetime
df_pila_i_sie['Perido Pago'] = pd.to_datetime(df_pila_i_sie['Perido Pago'], format='%Y-%m', errors='coerce')

# Para cada grupo, obtener el índice del registro con la fecha máxima
idx_max = df_pila_i_sie.groupby(['N° Identificación Aportante', 'Tp_Do', 'No_Do'])['Perido Pago'].idxmax()

# Seleccionar únicamente esos registros
df_pila_i_sie = df_pila_i_sie.loc[idx_max].reset_index(drop=True)

# Mostrar la cantidad de registros después de filtrar
print(f"Cantidad de registros después de filtrar: {len(df_pila_i_sie)}")

Cantidad de registros después de filtrar: 66031


In [24]:
# Seleccionar solo las columnas necesarias de df_Datos_Complementarios
df_Datos_Complementarios_subset = df_Datos_Complementarios[['N° Identificación Aportante', 'Dirección de Correspondencia', 'Teléfono', 'Correo Electrónico']]

# Realizar el merge para traer las columnas a df_pila_i_sie
df_pila_i_sie = df_pila_i_sie.merge(df_Datos_Complementarios_subset, on='N° Identificación Aportante', how='left')

print(df_pila_i_sie.columns)

Index(['Razón Social Aportante', 'N° Identificación Aportante', 'Perido Pago',
       'Fecha Pago', 'Tp_Do', 'No_Do', 'Tipo Cotizante', 'ING', 'RET',
       'Días Cotizados', 'Ingreso Base Cotización', 'Cotización Obligatoria',
       'Número Planilla', 'origen', '1A', '2A', '1N', '2N', 'Tp_Afiliado',
       'Departamento', 'Municipio', 'Estado_ADRES', 'Sisben',
       'Dirección de Correspondencia', 'Teléfono', 'Correo Electrónico'],
      dtype='object')


In [25]:
# Crear un nuevo dataframe con los valores únicos de las columnas especificadas
df_unique_aportantes = df_pila_i_sie[["Tp_Do", "No_Do"]].drop_duplicates()

# Mostrar las primeras filas del nuevo dataframe
print(f"Cantidad de registros únicos: {len(df_unique_aportantes)}")

Cantidad de registros únicos: 15868


In [26]:
print(f"Cantidad de registros realcionaes laborales #1: {len(df_Relaciones_Laborales_SIE)}")
# Convertir la columna 'fecha_ingreso' a tipo datetime
df_Relaciones_Laborales_SIE['fecha_ingreso'] = pd.to_datetime(df_Relaciones_Laborales_SIE['fecha_ingreso'], format='%Y-%m-%d')

# Asignar la fecha máxima de 'fecha_ingreso' a todo el grupo
df_Relaciones_Laborales_SIE['fecha_ingreso'] = df_Relaciones_Laborales_SIE.groupby(
    ['tipo_documento', 'numero_identificacion', 'tipo_documento_aportante', 'numero_identificacion_aportante']
)['fecha_ingreso'].transform('max')

# Eliminar duplicados manteniendo solo un registro por grupo
df_Relaciones_Laborales_SIE = df_Relaciones_Laborales_SIE.drop_duplicates(
    subset=['tipo_documento', 'numero_identificacion', 'tipo_documento_aportante', 'numero_identificacion_aportante']
)

# Mostrar las primeras filas del dataframe resultante
print(f"Cantidad de registros realcionaes laborales #2: {len(df_Relaciones_Laborales_SIE)}")

Cantidad de registros realcionaes laborales #1: 24724
Cantidad de registros realcionaes laborales #2: 24367


In [27]:
# Filtrar los registros en DF_MC_Adres que no están en df_unique_aportantes
df_sin_pagos = DF_MC_Adres.merge(df_unique_aportantes, on=["Tp_Do", "No_Do"], how="left", indicator=True)
df_sin_pagos = df_sin_pagos[df_sin_pagos["_merge"] == "left_only"].drop(columns=["_merge"])

# Filtrar las columnas [Tp_Afiliado= 'C' y Estado_ADRES= 'AC']
df_sin_pagos = df_sin_pagos[(df_sin_pagos['Tp_Afiliado'] == 'C') & (df_sin_pagos['Estado_ADRES'] == 'AC')]

# Mostrar la cantidad de registros filtrados
print(f"Cantidad de registros sin pagos: {len(df_sin_pagos)}")


Cantidad de registros sin pagos: 95


In [28]:
print(df_pila_i_sie.columns)
print(df_Relaciones_Laborales_SIE.columns)

Index(['Razón Social Aportante', 'N° Identificación Aportante', 'Perido Pago',
       'Fecha Pago', 'Tp_Do', 'No_Do', 'Tipo Cotizante', 'ING', 'RET',
       'Días Cotizados', 'Ingreso Base Cotización', 'Cotización Obligatoria',
       'Número Planilla', 'origen', '1A', '2A', '1N', '2N', 'Tp_Afiliado',
       'Departamento', 'Municipio', 'Estado_ADRES', 'Sisben',
       'Dirección de Correspondencia', 'Teléfono', 'Correo Electrónico'],
      dtype='object')
Index(['tipo_documento', 'numero_identificacion', 'tipo_documento_aportante',
       'numero_identificacion_aportante', 'razon_social', 'fecha_ingreso',
       'Unnamed: 6'],
      dtype='object')


In [29]:
df_Relaciones_Laborales_SIE = df_Relaciones_Laborales_SIE.rename(columns={
    'tipo_documento': 'Tp_Do',
    'numero_identificacion': 'No_Do',
    'tipo_documento_aportante': 'Tipo Documento Aportante',
    'numero_identificacion_aportante': 'N° Identificación Aportante',
    'razon_social': 'Razón Social Aportante'
})
print(df_Relaciones_Laborales_SIE.columns)
df_Relaciones_Laborales_SIE['N° Identificación Aportante'] = df_Relaciones_Laborales_SIE['N° Identificación Aportante'].astype(str)
df_pila_i_sie['N° Identificación Aportante'] = df_pila_i_sie['N° Identificación Aportante'].astype(str)

df_Relaciones_Laborales_SIE = df_Relaciones_Laborales_SIE.merge(df_pila_i_sie[['N° Identificación Aportante', 'Correo Electrónico']], on=['N° Identificación Aportante'], how='left')
print(df_Relaciones_Laborales_SIE.columns)

Index(['Tp_Do', 'No_Do', 'Tipo Documento Aportante',
       'N° Identificación Aportante', 'Razón Social Aportante',
       'fecha_ingreso', 'Unnamed: 6'],
      dtype='object')
Index(['Tp_Do', 'No_Do', 'Tipo Documento Aportante',
       'N° Identificación Aportante', 'Razón Social Aportante',
       'fecha_ingreso', 'Unnamed: 6', 'Correo Electrónico'],
      dtype='object')


In [30]:
# Convertir 'No_Do' en df_Relaciones_Laborales_SIE a str para que coincida con df_sin_pagos
df_Relaciones_Laborales_SIE['No_Do'] = df_Relaciones_Laborales_SIE['No_Do'].astype(str)
df_sin_pagos['No_Do'] = df_sin_pagos['No_Do'].astype(str)

# Realizar el merge para traer la columna 'fecha_ingreso' a df_sin_pagos
df_sin_pagos = df_sin_pagos.merge(df_Relaciones_Laborales_SIE[['Tp_Do', 'No_Do', 'Tipo Documento Aportante', 'N° Identificación Aportante', 'Razón Social Aportante', 'fecha_ingreso', 'Correo Electrónico']], on=['Tp_Do', 'No_Do'], how='left')
print(df_sin_pagos.columns)

Index(['Tp_Do', 'No_Do', '1A', '2A', '1N', '2N', 'Tp_Afiliado', 'Departamento',
       'Municipio', 'Estado_ADRES', 'Sisben', 'Tipo Documento Aportante',
       'N° Identificación Aportante', 'Razón Social Aportante',
       'fecha_ingreso', 'Correo Electrónico'],
      dtype='object')


In [31]:
print(f"Cantidad de registros sin pagos: {len(df_sin_pagos)}")
# Dejar valores únicos en df_sin_pagos según las columnas ['Tp_Do', 'No_Do']
df_sin_pagos = df_sin_pagos.drop_duplicates(subset=['Tp_Do', 'No_Do'])

# Mostrar las primeras filas del dataframe resultante para verificar
print(f"Cantidad de registros únicos en df_sin_pagos: {len(df_sin_pagos)}")
print(df_sin_pagos.columns)

Cantidad de registros sin pagos: 4184
Cantidad de registros únicos en df_sin_pagos: 95
Index(['Tp_Do', 'No_Do', '1A', '2A', '1N', '2N', 'Tp_Afiliado', 'Departamento',
       'Municipio', 'Estado_ADRES', 'Sisben', 'Tipo Documento Aportante',
       'N° Identificación Aportante', 'Razón Social Aportante',
       'fecha_ingreso', 'Correo Electrónico'],
      dtype='object')


In [32]:
# Convert No_Do to string type in both dataframes
df_sin_pagos['No_Do'] = df_sin_pagos['No_Do'].astype(str)
df_unique_aportantes['No_Do'] = df_unique_aportantes['No_Do'].astype(str)

# Now perform the merge
df_sin_pagos = df_sin_pagos.merge(df_unique_aportantes, on=["Tp_Do", "No_Do"], how="left", indicator=True)
df_sin_pagos = df_sin_pagos[df_sin_pagos["_merge"] == "left_only"].drop(columns=["_merge"])

# Filter records with required conditions
df_sin_pagos = df_sin_pagos[(df_sin_pagos['Tp_Afiliado'] == 'C') & (df_sin_pagos['Estado_ADRES'] == 'AC')]

# Show results
print(f"Cantidad de registros sin pagos: {len(df_sin_pagos)}")
print(df_sin_pagos.columns)


Cantidad de registros sin pagos: 95
Index(['Tp_Do', 'No_Do', '1A', '2A', '1N', '2N', 'Tp_Afiliado', 'Departamento',
       'Municipio', 'Estado_ADRES', 'Sisben', 'Tipo Documento Aportante',
       'N° Identificación Aportante', 'Razón Social Aportante',
       'fecha_ingreso', 'Correo Electrónico'],
      dtype='object')


In [33]:
# Add new columns to df_sin_pagos
df_sin_pagos['No_Do'] = df_sin_pagos['No_Do'].astype('int64')
df_sin_pagos['N° Identificación Aportante'] = df_sin_pagos['N° Identificación Aportante'].fillna(0).astype('int64')
df_sin_pagos['Tipo Cotizante'] = 0
df_sin_pagos['Tipo Cotizante'] = df_sin_pagos['Tipo Cotizante'].astype('int64')
df_sin_pagos['Dirección de Correspondencia'] = ""
df_sin_pagos['Perido Pago'] = pd.to_datetime("")
df_sin_pagos['Fecha Pago'] = ""

df_sin_pagos['ING'] = 0
df_sin_pagos['ING'] = df_sin_pagos['ING'].astype('int64')
df_sin_pagos['RET'] = 0
df_sin_pagos['RET'] = df_sin_pagos['RET'].astype('int64')
df_sin_pagos['Días Cotizados'] = 0
df_sin_pagos['Días Cotizados'] = df_sin_pagos['Días Cotizados'].astype('int64')
df_sin_pagos['Ingreso Base Cotización'] = 0
df_sin_pagos['Ingreso Base Cotización'] = df_sin_pagos['Ingreso Base Cotización'].astype('float64')
df_sin_pagos['Correcciones'] = ""
df_sin_pagos['Número Planilla'] = 0
df_sin_pagos['Número Planilla'] = df_sin_pagos['Número Planilla'].astype('int64')
df_sin_pagos['Departamento'] = df_sin_pagos['Departamento'].astype('int64')
df_sin_pagos['Departamento'] = df_sin_pagos['Departamento'].astype(str).str.zfill(2)
df_sin_pagos['Municipio'] = df_sin_pagos['Municipio'].astype('int64')
df_sin_pagos['Municipio'] = df_sin_pagos['Municipio'].astype(str).str.zfill(3)
df_sin_pagos['origen'] = "MC_ADRES"

df_pila_i_sie['Tipo Cotizante'] = df_pila_i_sie['Tipo Cotizante'].fillna(0).astype('int64')
df_pila_i_sie['ING'] = df_pila_i_sie['ING'].fillna(0).astype('int64')
df_pila_i_sie['RET'] = df_pila_i_sie['RET'].fillna(0).astype('int64')
df_pila_i_sie['Departamento'] = df_pila_i_sie['Departamento'].astype('int64')
df_pila_i_sie['Departamento'] = df_pila_i_sie['Departamento'].astype(str).str.zfill(2)
df_pila_i_sie['Municipio'] = df_pila_i_sie['Municipio'].astype('int64')
df_pila_i_sie['Municipio'] = df_pila_i_sie['Municipio'].astype(str).str.zfill(3)
df_pila_i_sie['fecha_ingreso'] = pd.to_datetime("")

In [34]:
# Reorganizar las columnas de df_sin_pagos en el orden especificado
column_order = ['Razón Social Aportante', 'Tipo Documento Aportante', 'N° Identificación Aportante', 'Dirección de Correspondencia', 'Correo Electrónico', 'Perido Pago', 'Fecha Pago', 'Tp_Do', 'No_Do', 'Tipo Cotizante', 'ING', 'RET', 'Días Cotizados', 'Ingreso Base Cotización', 'Correcciones', 'Número Planilla', 'origen', '1A', '2A', '1N', '2N', 'Tp_Afiliado', 'Departamento', 'Municipio', 'Estado_ADRES', 'Sisben', 'fecha_ingreso']
df_sin_pagos = df_sin_pagos[column_order]

In [35]:
print(f"Cantidad de registros sin pagos: {len(df_sin_pagos)}")
print(f"Cantidad de registros PAgos SIE pagos: {len(df_pila_i_sie)}")
df_pila_i_sie = pd.concat([df_pila_i_sie, df_sin_pagos], ignore_index=True)
print(f"Cantidad de registros Total: {len(df_pila_i_sie)}")

Cantidad de registros sin pagos: 95
Cantidad de registros PAgos SIE pagos: 80096
Cantidad de registros Total: 80191


In [36]:
# Ensure 'Perido Pago' is datetime
print(f"Cantidad de registros df_pila_i_sie: {len(df_pila_i_sie)}")

df_pila_i_sie['Perido Pago'] = pd.to_datetime(df_pila_i_sie['Perido Pago'], errors='coerce')

# Filter out records where RET = 1
df_pila_i_sie = df_pila_i_sie[
    (df_pila_i_sie['RET'] != 1) &
    ((df_pila_i_sie['Perido Pago'] < pd.to_datetime(V_Periodo_Actual)) | (df_pila_i_sie['Perido Pago'].isna()))
]
print(f"Cantidad de registros después de filtrar RET != 1: {len(df_pila_i_sie)}")

Cantidad de registros df_pila_i_sie: 80191
Cantidad de registros después de filtrar RET != 1: 22731


In [37]:
# Create new column 'Cartera' based on conditions
df_pila_i_sie['Cartera'] = 'Sin Pagos'  # Default value

# Convert Mora to datetime
mora_date = pd.to_datetime(Mora)
Dia_date = pd.to_datetime(Dia)

# Update values based on conditions for non-null Perido Pago
mask = df_pila_i_sie['Perido Pago'].notna()
df_pila_i_sie.loc[mask & (df_pila_i_sie['Perido Pago'] < mora_date), 'Cartera'] = 'mora'
df_pila_i_sie.loc[mask & (df_pila_i_sie['Perido Pago'] >= mora_date), 'Cartera'] = 'Aviso'
df_pila_i_sie.loc[mask & (df_pila_i_sie['Perido Pago'] >= Dia_date), 'Cartera'] = 'Al Dia'
print(f"Cantidad de registros df_pila_i_sie: {len(df_pila_i_sie)}")

Cantidad de registros df_pila_i_sie: 22731


In [38]:
# Convert column 31 to datetime format
#DF_Automatico_R1["Fecha_Efectiva"] = pd.to_datetime(DF_Automatico_R1[34], format='%d/%m/%Y')

# Sort by date in descending order and keep first record for each ID
#DF_Automatico_R1 = DF_Automatico_R1.sort_values(by=[2, 3, 34], ascending=[True, True, False])
#DF_Automatico_R1 = DF_Automatico_R1.drop_duplicates(subset=[2, 3], keep='first')

# Reset the index if needed
#DF_Automatico_R1 = DF_Automatico_R1.reset_index(drop=True)

In [39]:
# Create list of column names where column 31 is 'Fecha_R1' and others are 'col#'
#column_names = [f'col{i}' for i in range(len(DF_Automatico_R1.columns))]
#column_names[34] = 'Fecha_R1'
#column_names[2] = 'Tp_Do'
#column_names[3] = 'No_Do'

# Assign column names to dataframe
#DF_Automatico_R1.columns = column_names

In [40]:
# Merge df_pila_i_sie with DF_Automatico_R1 to get Fecha_R1
#df_pila_i_sie = df_pila_i_sie.merge(
#    DF_Automatico_R1[['Tp_Do', 'No_Do', 'Fecha_R1']], 
#    on=['Tp_Do', 'No_Do'], 
#    how='left'
#)
#print(f"Cantidad de registros df_pila_i_sie: {len(df_pila_i_sie)}")
#print(df_pila_i_sie.columns)

In [41]:
# Sort by date in descending order and keep first record for each ID
df_Relaciones_Laborales_SIE = df_Relaciones_Laborales_SIE.sort_values(
    by=['Tp_Do', 'No_Do', 'fecha_ingreso'], 
    ascending=[True, True, False]
)

# Drop duplicates keeping first record (which will be the one with max date due to sort)
df_Relaciones_Laborales_SIE = df_Relaciones_Laborales_SIE.drop_duplicates(
    subset=['Tp_Do', 'No_Do'], 
    keep='first'
)

# Reset the index if needed
df_Relaciones_Laborales_SIE = df_Relaciones_Laborales_SIE.reset_index(drop=True)

print(f"Cantidad de registros después de eliminar duplicados: {len(df_Relaciones_Laborales_SIE)}")

Cantidad de registros después de eliminar duplicados: 21594


In [42]:
# Convertir la columna 'Perido Pago' al formato datetime con el formato dd/mm/yyyy
df_pila_i_sie['Perido Pago'] = pd.to_datetime(df_pila_i_sie['Perido Pago'], format='%d/%m/%Y', errors='coerce')

# Definir el rango de fechas deseado
start_date = pd.to_datetime('2023-01-01')
end_date   = pd.to_datetime('2026-12-31')

# Filtrar: conservar los registros cuyo 'Perido Pago' esté vacío o esté entre las fechas definidas
mask = df_pila_i_sie['Perido Pago'].isna() | ((df_pila_i_sie['Perido Pago'] >= start_date) & (df_pila_i_sie['Perido Pago'] <= end_date))
df_pila_i_sie = df_pila_i_sie[mask]

print(f"Cantidad de registros después del filtrado: {len(df_pila_i_sie)}")

Cantidad de registros después del filtrado: 20324


In [43]:
# Lista de columnas que contienen fechas (ajusta según tu DataFrame)
columnas_fecha = ['Perido Pago', 'fecha_ingreso']  # Reemplázalas con los nombres reales

# Convertir las columnas de fecha al formato DD/MM/YYYY
for col in columnas_fecha:
    df_pila_i_sie[col] = pd.to_datetime(df_pila_i_sie[col], errors='coerce').dt.strftime('%d/%m/%Y')

In [44]:
df_pila_i_sie.to_excel(R_Salida_Pila_SIE_i, index=False, engine='openpyxl')
#df_Relaciones_Laborales_SIE.to_csv(R_Salida_Relaciones_Laborales, sep=',', index=False, encoding='ANSI')